In [9]:
import pandas as pd

In [10]:
df = pd.read_csv('ball-ref-03.csv')
print(df)

     Unnamed: 0   Rk       Player_Name Pos  Age   Tm   G  GS    MP   FG  ...  \
0             0    1  Precious Achiuwa   C   23  TOR  55  12  20.7  3.6  ...   
1             1    2      Steven Adams   C   29  MEM  42  42  27.0  3.7  ...   
2             2    3       Bam Adebayo   C   25  MIA  75  75  34.6  8.0  ...   
3             3    4      Ochai Agbaji  SG   22  UTA  59  22  20.5  2.8  ...   
4             4    5      Santi Aldama  PF   22  MEM  77  20  21.8  3.2  ...   
..          ...  ...               ...  ..  ...  ...  ..  ..   ...  ...  ...   
674         674  535    Thaddeus Young  PF   34  TOR  54   9  14.7  2.0  ...   
675         675  536        Trae Young  PG   24  ATL  73  73  34.8  8.2  ...   
676         676  537    Omer Yurtseven   C   24  MIA   9   0   9.2  1.8  ...   
677         677  538       Cody Zeller   C   30  MIA  15   2  14.5  2.5  ...   
678         678  539       Ivica Zubac   C   25  LAC  76  76  28.6  4.3  ...   

     DRB   TRB   AST  STL  BLK  TOV   P

In [11]:
df.to_csv('ball-ref-02.csv')

In [12]:
pip install PandasBasketball

Note: you may need to restart the kernel to use updated packages.


In [13]:
from PandasBasketball import pandasbasketball as pb

In [14]:
import requests
from bs4 import BeautifulSoup

from PandasBasketball.stats import player_stats, team_stats, player_gamelog, n_days
from PandasBasketball.errors import StatusCode404, TableNonExistent 

BASE_URL = "https://www.basketball-reference.com"

In [15]:
def get_player_gamelog(player, season, player_name, playoffs=False):
    """
    Returns all of the player's games in specified season as a data frame
    \tKeyword arguments:
    \t\tplayer -- the player's url code
    \t\tseason -- the season (e.g. 2018 for the 2017-18 season)
    """

    url = BASE_URL + f"/players/{player[0]}/{player}/gamelog/{season}"
    r = requests.get(url)

    if r.status_code == 404:
        raise StatusCode404
    else:
        gamelog_df = player_gamelog(r, playoffs=playoffs)

        # Remove rows with more than half of the values missing
        gamelog_df = gamelog_df.dropna(how='any')
        # Insert the player's name as the first column of the DataFrame
        gamelog_df.insert(0, 'Player Name', player_name)

        return gamelog_df


In [16]:
player = 'achiupr01'
season = 2023
player_name = 'Precious Achiuwa'

gamelog_precious = get_player_gamelog(player, season, player_name)
print(gamelog_precious)


         Player Name   G        Date     Age   Tm     Opp          GS     MP  \
Rk                                                                             
1   Precious Achiuwa   1  2022-10-19  23-030  TOR     CLE   W (+3)  0  17:56   
2   Precious Achiuwa   2  2022-10-21  23-032  TOR  @  BRK   L (-4)  0  17:29   
3   Precious Achiuwa   3  2022-10-22  23-033  TOR  @  MIA   L (-3)  0  33:32   
4   Precious Achiuwa   4  2022-10-24  23-035  TOR  @  MIA   W (+8)  0  33:43   
5   Precious Achiuwa   5  2022-10-26  23-037  TOR     PHI  W (+10)  0  21:18   
..               ...  ..         ...     ...  ... ..  ...      ... ..    ...   
78  Precious Achiuwa  51  2023-04-02  23-195  TOR  @  CHO  W (+20)  0  12:32   
79  Precious Achiuwa  52  2023-04-04  23-197  TOR  @  CHO  W (+20)  0  14:55   
80  Precious Achiuwa  53  2023-04-05  23-198  TOR  @  BOS   L (-4)  0  19:45   
81  Precious Achiuwa  54  2023-04-07  23-200  TOR  @  BOS  L (-19)  0  23:32   
82  Precious Achiuwa  55  2023-04-09  23

In [17]:
import time

In [18]:
def get_player_gamelog(df, season, playoffs=False):
    """
    Returns all players' games in specified season as a single data frame
    \tKeyword arguments:
    \t\tdf -- the DataFrame with columns 'Player' and 'Player_Name'
    \t\tseason -- the season 
    """

    all_gamelogs = []

    for i, row in df.iterrows():
        player = row['Player']
        player_name = row['Player_Name']

        url = BASE_URL + f"/players/{player[0]}/{player}/gamelog/{season}"
        r = requests.get(url)

        if r.status_code == 404:
            raise StatusCode404
        else:
            gamelog_df = player_gamelog(r, playoffs=playoffs)

            
            gamelog_df = gamelog_df.dropna(how='any', axis=0)


            gamelog_df.insert(0, 'Player Name', player_name)

            all_gamelogs.append(gamelog_df)
            
         
            time.sleep(2)

    all_gamelogs_df = pd.concat(all_gamelogs, ignore_index=True)

    return all_gamelogs_df


In [20]:
all_game_log = get_player_gamelog(df, season, playoffs=False)
print(all_game_log)

            Player Name   G        Date     Age   Tm     Opp          GS  \
0      Precious Achiuwa   1  2022-10-19  23-030  TOR     CLE   W (+3)  0   
1      Precious Achiuwa   2  2022-10-21  23-032  TOR  @  BRK   L (-4)  0   
2      Precious Achiuwa   3  2022-10-22  23-033  TOR  @  MIA   L (-3)  0   
3      Precious Achiuwa   4  2022-10-24  23-035  TOR  @  MIA   W (+8)  0   
4      Precious Achiuwa   5  2022-10-26  23-037  TOR     PHI  W (+10)  0   
...                 ...  ..         ...     ...  ... ..  ...      ... ..   
51191       Ivica Zubac  72  2023-03-31  26-013  LAC  @  MEM  L (-14)  1   
51192       Ivica Zubac  73  2023-04-01  26-014  LAC  @  NOP   L (-8)  1   
51193       Ivica Zubac  74  2023-04-05  26-018  LAC     LAL   W (+7)  1   
51194       Ivica Zubac  75  2023-04-08  26-021  LAC     POR  W (+11)  1   
51195       Ivica Zubac  76  2023-04-09  26-022  LAC  @  PHO   W (+5)  1   

          MP  ... DRB TRB AST STL BLK TOV PF PTS  GmSc  +/-  
0      17:56  ...   4   5

In [21]:
all_game_log.to_csv('ball-ref-05.csv')

In [22]:
all_game_log.tail(20)

,Player Name,G,Date,Age,Tm,,Opp,,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
51176,Ivica Zubac,,2023-02-26,25-345,LAC,@,DEN,L (-10),,,...,,,,,,,,,,
51177,Ivica Zubac,60,2023-02-28,25-347,LAC,,MIN,L (-7),1,28:27,...,6,9,1,1,2,2,3,12,12.9,+3
51178,Ivica Zubac,,2023-03-02,25-349,LAC,@,GSW,L (-24),,,...,,,,,,,,,,
51179,Ivica Zubac,,2023-03-03,25-350,LAC,@,SAC,L (-1),,,...,,,,,,,,,,
51180,Ivica Zubac,61,2023-03-05,25-352,LAC,,MEM,W (+6),1,17:34,...,5,9,2,0,2,1,2,6,9.0,-14
51181,Ivica Zubac,62,2023-03-08,25-355,LAC,,TOR,W (+8),1,27:21,...,7,8,1,0,2,1,3,17,16.5,-2
51182,Ivica Zubac,63,2023-03-11,25-358,LAC,,NYK,W (+11),1,27:20,...,4,6,0,0,1,0,3,8,7.5,+6
51183,Ivica Zubac,64,2023-03-15,25-362,LAC,,GSW,W (+8),1,30:26,...,11,16,2,0,1,1,2,19,22.5,+10
51184,Ivica Zubac,65,2023-03-18,26-000,LAC,,ORL,L (-5),1,27:14,...,9,16,0,0,2,2,2,16,13.8,-2
51185,Ivica Zubac,66,2023-03-19,26-001,LAC,@,POR,W (+15),1,33:53,...,7,10,2,1,0,0,1,14,14.5,+26
